In [1]:
import warnings
warnings.filterwarnings('ignore')
import main

In [2]:
search_engine_2 = main.SearchEngineElasticSearch(init_path=r'C:\Users\sergiojesus\Desktop\Document_Clustering\CV', index_name='cv')

In [4]:
% time results = search_engine_2.query_database('design', summary=False, max_size=10)

Results:


,File,Link,Score
0,David Martínez Moll_10-2017,cvs/David Martínez Moll_10-2017.pdf,4.4541
1,Tiago Oliveira,cvs/Tiago Oliveira.pdf,4.4492
2,Florentino Gutierrez Prieto,cvs/Florentino Gutierrez Prieto.pdf,4.43541
3,Ashokan Gopal,cvs/Ashokan Gopal.pdf,4.43486
4,Gaspar Quelhas Lima Tameeris,cvs/Gaspar Quelhas Lima Tameeris.pdf,4.432
5,Carlos Jorge Laruça_03_2017,cvs/Carlos Jorge Laruça_03_2017.pdf,4.41694
6,Nuno Miguel da Silva_03_2017,cvs/Nuno Miguel da Silva_03_2017.pdf,4.41294
7,Luís Almeida,cvs/Luís Almeida.pdf,4.41193
8,Nuno Ferreira_03_2017,cvs/Nuno Ferreira_03_2017.pdf,4.40515
9,José Júlio Ribeiro Rodrigues_03_2017,cvs/José Júlio Ribeiro Rodrigues_03_2017.pdf,4.39837


Wall time: 38.9 ms
